# [Advent of Code 2018: Day 2](https://adventofcode.com/2018/day/2)

<h2>--- Day 2: Inventory Management System ---</h2>
<p>You stop falling through time, catch your breath, and check the screen on the device. "Destination reached. Current Year: 1518. Current Location: North Pole Utility Closet 83N10." You made it! Now, to find those anomalies.</p>
<p>Outside the utility closet, you hear footsteps and a voice. "...I'm not sure either. But now that <span title="This is, in fact, roughly when chimneys became common in houses.">so many people have chimneys</span>, maybe he could sneak in that way?" Another
    voice responds, "Actually, we've been working on a new kind of <em>suit</em> that would let him fit through tight spaces like that. But, I heard that a few days ago, they lost the prototype fabric, the design plans, everything! Nobody on the team
    can even seem to remember important details of the project!"</p>
<p>"Wouldn't they have had enough fabric to fill several boxes in the warehouse? They'd be stored together, so the box IDs should be similar. Too bad it would take forever to search the warehouse for <em>two similar box IDs</em>..." They walk too far away
    to hear any more.</p>
<p>Late at night, you sneak to the warehouse - who knows what kinds of paradoxes you could cause if you were discovered - and use your fancy wrist device to quickly scan every box and produce a list of the likely candidates (your puzzle input).</p>
<p>To make sure you didn't miss any, you scan the likely candidate boxes again, counting the number that have an ID containing <em>exactly two of any letter</em> and then separately counting those with <em>exactly three of any letter</em>. You can multiply
    those two counts together to get a rudimentary <a href="https://en.wikipedia.org/wiki/Checksum">checksum</a> and compare it to what your device predicts.</p>
<p>For example, if you see the following box IDs:</p>
<ul>
    <li><code>abcdef</code> contains no letters that appear exactly two or three times.</li>
    <li><code>bababc</code> contains two <code>a</code> and three <code>b</code>, so it counts for both.</li>
    <li><code>abbcde</code> contains two <code>b</code>, but no letter appears exactly three times.</li>
    <li><code>abcccd</code> contains three <code>c</code>, but no letter appears exactly two times.</li>
    <li><code>aabcdd</code> contains two <code>a</code> and two <code>d</code>, but it only counts once.</li>
    <li><code>abcdee</code> contains two <code>e</code>.</li>
    <li><code>ababab</code> contains three <code>a</code> and three <code>b</code>, but it only counts once.</li>
</ul>
<p>Of these box IDs, four of them contain a letter which appears exactly twice, and three of them contain a letter which appears exactly three times. Multiplying these together produces a checksum of <code>4 * 3 = 12</code>.</p>
<p><em>What is the checksum</em> for your list of box IDs?</p>

In [1]:
import os
import unittest

def calc_id_checksums(id_list, deliminator = '\n', verbose = False):
    
    if type(id_list) is str:
        id_list = id_list.split(deliminator)
    
    num_char_x2 = 0
    num_char_x3 = 0

    for id in id_list:
        chars_counted = []
        num_char_x2_done = False
        num_char_x3_done = False
        for char in id:
            if char not in chars_counted:
                chars_counted.append(char)
            
                if (id.count(char) is 2) and not num_char_x2_done:
                    num_char_x2 += 1
                    num_char_x2_done = True
                    
                if id.count(char) is 3 and not num_char_x3_done:
                    num_char_x3 += 1
                    num_char_x3_done = True
        
        if verbose:
            print(f'Chars x2: {num_char_x2}, x3: {num_char_x3}')
        
    return num_char_x2 * num_char_x3

class TestBasic(unittest.TestCase):
    def test_part_one(self):
        data = [(['abcdef','bababc','abbcde','abcccd','aabcdd','abcdee','ababab'],12)]
        for input, ans in data:
            self.assertEqual(calc_id_checksums(id_list = input, verbose = True), ans)
        
        
unittest.main(argv=[""], exit=False)

.

Chars x2: 0, x3: 0
Chars x2: 1, x3: 1
Chars x2: 2, x3: 1
Chars x2: 2, x3: 2
Chars x2: 3, x3: 2
Chars x2: 4, x3: 2
Chars x2: 4, x3: 3



----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


In [2]:
with open(os.path.join("inputs", "input_d02.txt")) as file:
    data = file.read()
    
calc_id_checksums(data)

4920

<h2 id="part2">--- Part Two ---</h2>
<p>Confident that your list of box IDs is complete, you're ready to find the boxes full of prototype fabric.</p>
<p>The boxes will have IDs which differ by exactly one character at the same position in both strings. For example, given the following box IDs:</p>
<pre><code>abcde
fghij
klmno
pqrst
fguij
axcye
wvxyz
</code></pre>
<p>The IDs <code>abcde</code> and <code>axcye</code> are close, but they differ by two characters (the second and fourth). However, the IDs <code>fghij</code> and <code>fguij</code> differ by exactly one character, the third (<code>h</code> and <code>u</code>).
    Those must be the correct boxes.</p>
<p><em>What letters are common between the two correct box IDs?</em> (In the example above, this is found by removing the differing character from either ID, producing <code>fgij</code>.)</p>

In [3]:
import difflib

def find_common_ids(id_list, num_diffs = 1, deliminator = '\n', verbose = False):
    
    if type(id_list) is str:
        id_list = id_list.split(deliminator)
        
    id_len = len(id_list[0])
    expected_ratio = (id_len-num_diffs)/id_len
    
    if verbose:
        print(f'Expected match ratio: {expected_ratio}')
    
    for id1 in id_list:
        for id2 in id_list:
            seq_matcher = difflib.SequenceMatcher(None, id1, id2)
            id_match_ratio = seq_matcher.ratio()
            if verbose:
                print(f'id1: {id1}, id2: {id2} match ratio: {id_match_ratio}')
            if id_match_ratio == expected_ratio:
                if verbose:
                    print(f'id1: {id1} matches id2: {id2} with the desired {id_match_ratio} match ratio')

                for opcode, id1s, id1e, id2s, id2e in seq_matcher.get_opcodes():
                    if opcode is 'replace':
                        matching_chars = id1[:id1s]+id1[id1e:]
                        return matching_chars
                
    return None

class TestBasic(unittest.TestCase):
    def test_part_one(self):
        data = [(['abcde','fghij','klmno','pqrst','fguij','axcye','wvxyz'],'fgij')]
        for input, ans in data:
            self.assertEqual(find_common_ids(id_list = input, verbose = True), ans)        
        
unittest.main(argv=[""], exit=False)

.

Expected match ratio: 0.8
id1: abcde, id2: abcde match ratio: 1.0
id1: abcde, id2: fghij match ratio: 0.0
id1: abcde, id2: klmno match ratio: 0.0
id1: abcde, id2: pqrst match ratio: 0.0
id1: abcde, id2: fguij match ratio: 0.0
id1: abcde, id2: axcye match ratio: 0.6
id1: abcde, id2: wvxyz match ratio: 0.0
id1: fghij, id2: abcde match ratio: 0.0
id1: fghij, id2: fghij match ratio: 1.0
id1: fghij, id2: klmno match ratio: 0.0
id1: fghij, id2: pqrst match ratio: 0.0
id1: fghij, id2: fguij match ratio: 0.8
id1: fghij matches id2: fguij with the desired 0.8 match ratio



----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [4]:
find_common_ids(id_list = data)

'fonbwmjquwtapeyzikghtvdxl'